# REST ChatBot

Advantages of using SK over vanilla REST or standar OpenAI SDK:

- HttpClient handling and built-in retry logic
- Pre-defined models (Prompt & Completion)
- Process prompts as pipes
- Prompts and prompt templates (semantic functions)
- Process native functions as semantic functions


## Load the required .NET packages and supporting classes

In [ ]:
using System;
using System.Net.Http;
using System.Net.Http.Headers;
using System.Text;
using System.Text.Json;
using System.Text.Json.Serialization;
using dotenv.net;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

#r "nuget: dotenv.net"

#!import Models/Models.cs
#!import Utils/HttpUtils.cs

## Load the OpenAI variables from environment variables or an .env file

In [ ]:
DotEnv.Load();
var uri = Environment.GetEnvironmentVariable("GPT_OPENAI_FULL_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
Console.WriteLine($"Using deployment at: {uri} with key {apiKey.Substring(0, 5)}...");

## Get a HttpClient instance

**> Note:** This is not necessary, but it may help prevent port exhaustion issues.

In [ ]:
// Note: This may be good practice depending on your use case. Otherwise, an HttpClient will be created for you.
var client = new HttpClient();
client.DefaultRequestHeaders.Accept.Add(new MediaTypeWithQualityHeaderValue("application/json"));
client.DefaultRequestHeaders.Add("api-key", $"{apiKey}");

## Call the bot for the first time

In [ ]:
var DefaultSystemMessage = "You are a general assitant.";
var chatHistory = new List<Message>();
chatHistory.Add(new Message("system", DefaultSystemMessage));
var ans = await GetGptCompletionAsync(client,uri,chatHistory);
Console.WriteLine(ans.Item1 + "\n");


## Enter the while loop and process the user and system messages

In [ ]:
while (true) {
    var userMessage = await InteractiveKernel.GetInputAsync("Your message");
    Console.WriteLine($"User: {userMessage}");
    switch(userMessage) {
        case "history":
            foreach (var message in chatHistory) {
                Console.WriteLine(message.Role+ ": " + message.Content);
            }
            continue;
        default:
            chatHistory.Add(new Message("user", userMessage));
            ans = await GetGptCompletionAsync(client,uri,chatHistory);
            Console.WriteLine(ans.Item1 + "\n");
            chatHistory.Add(new Message("system", ans.Item1));
            break;
    }        
}